In [3]:
from py2neo import Graph
from bs4 import BeautifulSoup
import pywikibot
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
import wikipedia
import spacy
nlp = spacy.load("en_core_web_sm")
import gensim
from collections import defaultdict
from time import sleep
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 

/anaconda3/lib/python3.6/site-packages/pywikibot/tools/__init__.py:2023: UserWarning: File /Users/curtis/Desktop/Wikipedia-Graph-Topic-Suggestion/pywikibot.lwp had 644 mode; converted to 600 mode.
  warn(warn_str.format(filename, st_mode - stat.S_IFREG, mode))


In [2]:
graph = Graph(password="kshen3778")

In [3]:
graph.run("MATCH (n:Category) RETURN n LIMIT 25").data()

[{'n': (_5214855:Category:Page {id: 690070, isNew: false, isRedirect: false, title: 'Futurama'})},
 {'n': (_5214856:Category:Page {id: 690451, isNew: false, isRedirect: false, title: 'World_War_II'})},
 {'n': (_5214857:Category:Page {id: 690571, isNew: false, isRedirect: false, title: 'Programming_languages'})},
 {'n': (_5214858:Category:Page {id: 690578, isNew: false, isRedirect: false, title: 'Professional_wrestling'})},
 {'n': (_5214859:Category:Page {id: 690637, isNew: false, isRedirect: false, title: 'Algebra'})},
 {'n': (_5214860:Category:Page {id: 690649, isNew: false, isRedirect: false, title: 'Anime'})},
 {'n': (_5214861:Category:Page {id: 690672, isNew: false, isRedirect: false, title: 'Abstract_algebra'})},
 {'n': (_5214862:Category:Page {id: 690747, isNew: false, isRedirect: false, title: 'Mathematics'})},
 {'n': (_5214863:Category:Page {id: 690777, isNew: false, isRedirect: false, title: 'Linear_algebra'})},
 {'n': (_5214864:Category:Page {id: 690803, isNew: false, isRedir

In [20]:
graph.run("MATCH (n:Category { title: 'Cold_War' }) RETURN n").data()

[{'n': (_5214918:Category:Page {id: 691614, isNew: false, isRedirect: false, title: 'Cold_War'})}]

In [55]:
#get all sub cat
cats = graph.run("MATCH (n:Category{title:'Constraint_logic_programming'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [56]:
len(cats)

2

In [57]:
cats

[{'p': (_6124108:Category:Page {id: 22968506, isNew: false, isRedirect: false, title: 'Constraint_programming'})},
 {'p': (_5594005:Category:Page {id: 1188828, isNew: false, isRedirect: false, title: 'Logic_programming'})}]

In [27]:
getBadCategories("Artificial_intelligence")

['Areas_of_computer_science',
 'Cognitive_science',
 'Computational_neuroscience',
 'Cybernetics',
 'Emerging_technologies',
 'Formal_sciences',
 'Futurology',
 'Intelligence_by_type',
 'Personhood',
 'Technology_in_society',
 'Unsolved_problems_in_computer_science']

In [63]:
#check if a specific sub category references the select category an two way relationship (which we don't want)
subs = graph.run("MATCH (n:Category{title:'Emerging_technologies'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [64]:
subs

[]

In [4]:
#verify if category actually exists on online Wikipedia
def categoryExist(name):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + name)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        return True
    else:
        return False

#Use beautifulsoup to scrape the bottom categories of a parent category (these we don't want)
def getBadCategories(parent):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + parent)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        to_return = []
        for item in div.findAll("li"):
            to_return.append("_".join(item.get_text().split(" ")))
        return to_return
    else:
        #parent does not exist
        return []

#get all immediate Wikipedia subcategories of main minus two way connections

# def getAllSubCat(parent):
#     all_cats = graph.run("MATCH (n:Category{title:'" + parent + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
#     bad_cats = getBadCategories(parent)
#     filtered = []
#     for category in all_cats:
#         cat_title = category['p']["title"]
#         if (not (cat_title in bad_cats)):
#             filtered.append(cat_title)
#     #search all these categories to see if any of them match the other way (if so, we remove)
# #     filtered = []
# #     for category in all_cats:
# #         cat_title = category['p']["title"]
# #         subs = graph.run("MATCH (n:Category{title:'" + cat_title + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
# #         found = False
# #         for category_2 in subs:
# #             sub_title = category_2['p']["title"]
# #             if(sub_title == parent):
# #                 found = True
# #         if (not found):
# #             filtered.append(cat_title)
    
#     return filtered

def getSubCategories(name):
    site = pywikibot.Site()
    gen = pywikibot.Category(site,'Category:' + name).subcategories(recurse=False)
    subcats = []
    for item in gen:
        subcats.append("_".join(item.aslink().split(":")[1][:-2].split(" ")))
    return subcats  

In [11]:
getAllSubCat("Works_about_personality")
#getBadCategories("Constraint_logic_programming")

['Borderline_personality_disorder_in_fiction',
 'Histrionic_personality_disorder_in_fiction']

In [5]:
blacklist = ["July_events"] #nodes that should not be visited
all_nodes = []
all_relations = [] #child parent
visited = []
def buildGraph(root, path):
    path = path + "/" + root
    print("Current Path: " + path)
    print()
    if((not categoryExist(root)) or (root in blacklist)):
        print("Category DNE or is BLACKLISTED")
        print()
        return
    if(root in visited):
        print("VISITED ALREADY")
        print()
        return
    
    visited.append(root)
    
    #add the node
    all_nodes.append(root)
    
    #get all subcategories
    sub_cat = getSubCategories(root)
    if(len(sub_cat) == 0): #base case
        return

    
    #add all the relationships
    for item in sub_cat:
        #check if this relationship already exists (cycle)
        relation = item + " " + root
        if(relation in all_relations):
            print("Relation already exists: ", relation)
            print()
            return
        all_relations.append(item + " " + root)
   
    #recurse on all sub nodes
    for item in sub_cat:
        buildGraph(item, path)


In [6]:
buildGraph("Machine_learning", "")

Current Path: /Machine_learning

Current Path: /Machine_learning/Applied_machine_learning

Current Path: /Machine_learning/Applied_machine_learning/AlphaGo

Current Path: /Machine_learning/Artificial_neural_networks

Current Path: /Machine_learning/Artificial_neural_networks/Deep_learning

Current Path: /Machine_learning/Artificial_neural_networks/Neural_network_software

Current Path: /Machine_learning/Bayesian_networks

Current Path: /Machine_learning/Classification_algorithms

Current Path: /Machine_learning/Classification_algorithms/Artificial_neural_networks

VISITED ALREADY

Current Path: /Machine_learning/Classification_algorithms/Decision_trees

Current Path: /Machine_learning/Classification_algorithms/Ensemble_learning

Current Path: /Machine_learning/Cluster_analysis

Current Path: /Machine_learning/Cluster_analysis/Cluster_analysis_algorithms

Current Path: /Machine_learning/Cluster_analysis/Clustering_criteria

Current Path: /Machine_learning/Computational_learning_theory



In [7]:
all_nodes

['Machine_learning',
 'Applied_machine_learning',
 'AlphaGo',
 'Artificial_neural_networks',
 'Deep_learning',
 'Neural_network_software',
 'Bayesian_networks',
 'Classification_algorithms',
 'Decision_trees',
 'Ensemble_learning',
 'Cluster_analysis',
 'Cluster_analysis_algorithms',
 'Clustering_criteria',
 'Computational_learning_theory',
 'Artificial_intelligence_conferences',
 'Signal_processing_conferences',
 'Data_mining_and_machine_learning_software',
 'Social_network_analysis_software',
 'Datasets_in_machine_learning',
 'Datasets_in_computer_vision',
 'Dimension_reduction',
 'Factor_analysis',
 'Evolutionary_algorithms',
 'Gene_expression_programming',
 'Genetic_algorithms',
 'Artificial_immune_systems',
 'Genetic_programming',
 'Nature-inspired_metaheuristics',
 'Inductive_logic_programming',
 'Kernel_methods_for_machine_learning',
 'Support_vector_machines',
 'Latent_variable_models',
 'Structural_equation_models',
 'Learning_in_computer_vision',
 'Log-linear_models',
 'Loss_

In [6]:
len(all_relations)

61

In [40]:
site = pywikibot.Site('en', 'wikipedia')  # The site we want to run our bot on
#page = pywikibot.Page(site, 'Support_vector_machines')
#page.text

page = wikipedia.page("Markov_networks")
page.content

"In the domain of physics and probability, a Markov random field (often abbreviated as MRF), Markov network or undirected graphical model is a set of random variables having a Markov property described by an undirected graph. In other words, a random field is said to be a Markov random field if it satisfies Markov properties.\nA Markov network or MRF is similar to a Bayesian network in its representation of dependencies; the differences being that Bayesian networks are directed and acyclic, whereas Markov networks are undirected and may be cyclic. Thus, a Markov network can represent certain dependencies that a Bayesian network cannot (such as cyclic dependencies); on the other hand, it can't represent certain dependencies that a Bayesian network can (such as induced dependencies). The underlying graph of a Markov random field may be finite or infinite.\nWhen the joint probability density of the random variables is strictly positive, it is also referred to as a Gibbs random field, beca

In [8]:
#save graph to files
fileName = "data/ML"
with open(fileName + '_nodes.txt', 'w') as f:
    for item in all_nodes:
        f.write("%s\n" % item)

with open(fileName + '_relations.txt', 'w') as f:
    for item in all_relations:
        f.write("%s\n" % item)        

In [10]:
def preprocess(x):
    x = gensim.utils.simple_preprocess(x)
    x = " ".join(x)
    
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords]
    x = preprocess_string(x, CUSTOM_FILTERS)
    return x

In [11]:
#Extract wikipedia articles (to convert into feature vectors)
file = open(fileName + '_nodes.txt', "r")
articles_cleaned = []
freq_threshold = 3
for line in file:
    #sleep(10)
    print("==" + line + "==")
#     all_good = True
#     while(True):
    try:
        search = wikipedia.search(line.rstrip("\n"))[0]
        page = wikipedia.page(search)
    except wikipedia.DisambiguationError as e:
        page = wikipedia.page(e.options[0])
    except wikipedia.WikipediaException as e:
        all_good = False
    except Exception:
        articles_cleaned.append("")
        continue;
#         if all_good:
#             break;
    #page = wikipedia.page(wikipedia.search(line.rstrip("\n"))[0])
    content = page.content
    cleaned = preprocess(content) #strip punctuations and stopwords and weird characters
    cleaned = " ".join(cleaned)
    doc = nlp(cleaned)
    lemm = [token.lemma_ for token in doc]
    d = defaultdict(int)
    for item in lemm:
        d[item] += 1
    tokens=[key for key,value in d.items() if value>freq_threshold] #all words with frequency of more than some number
    texts = [word for word in lemm if word in tokens]
    texts = " ".join(texts)
    articles_cleaned.append(texts)
    #print(texts)
file.close()

==Machine_learning
==
==Applied_machine_learning
==
==AlphaGo
==
==Artificial_neural_networks
==
==Deep_learning
==
==Neural_network_software
==
==Bayesian_networks
==
==Classification_algorithms
==
==Decision_trees
==
==Ensemble_learning
==
==Cluster_analysis
==
==Cluster_analysis_algorithms
==
==Clustering_criteria
==
==Computational_learning_theory
==
==Artificial_intelligence_conferences
==
==Signal_processing_conferences
==
==Data_mining_and_machine_learning_software
==
==Social_network_analysis_software
==
==Datasets_in_machine_learning
==
==Datasets_in_computer_vision
==
==Dimension_reduction
==
==Factor_analysis
==
==Evolutionary_algorithms
==
==Gene_expression_programming
==
==Genetic_algorithms
==
==Artificial_immune_systems
==
==Genetic_programming
==
==Nature-inspired_metaheuristics
==
==Inductive_logic_programming
==
==Kernel_methods_for_machine_learning
==
==Support_vector_machines
==
==Latent_variable_models
==
==Structural_equation_models
==
==Learning_in_computer_visio

In [12]:
len(articles_cleaned)

52

In [13]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(articles_cleaned)
print(vectorizer.get_feature_names())
print(vectors.shape)

['aaai', 'ab', 'ability', 'able', 'abstract', 'access', 'account', 'accuracy', 'achieve', 'acm', 'acquire', 'acquisition', 'act', 'action', 'activation', 'actual', 'acyclic', 'adaptation', 'adaptive', 'add', 'addition', 'address', 'adjust', 'advertising', 'agent', 'ai', 'aid', 'aim', 'ais', 'al', 'algorithm', 'algorithmic', 'align', 'allow', 'alpha', 'alphago', 'alphazero', 'analysis', 'analyze', 'ann', 'anneal', 'annotation', 'anns', 'annual', 'anomaly', 'answer', 'ant', 'anti', 'antibody', 'ap', 'application', 'apply', 'approach', 'appropriate', 'approx', 'approximate', 'approximation', 'aptitude', 'arbitrary', 'architecture', 'area', 'argmax', 'argmin', 'argue', 'argument', 'arise', 'arity', 'array', 'arrive', 'arrow', 'art', 'artificial', 'asr', 'assess', 'assign', 'assignment', 'associate', 'association', 'assume', 'assumption', 'assure', 'attempt', 'attribute', 'auditory', 'author', 'automate', 'automatic', 'automatically', 'autonomous', 'available', 'average', 'averaging', 'awar

In [14]:
article_names = []
file = open(fileName + '_nodes.txt', "r")
for line in file:
    article_names.append(line.rstrip("\n"))
file.close()

In [15]:
article_names

['Machine_learning',
 'Applied_machine_learning',
 'AlphaGo',
 'Artificial_neural_networks',
 'Deep_learning',
 'Neural_network_software',
 'Bayesian_networks',
 'Classification_algorithms',
 'Decision_trees',
 'Ensemble_learning',
 'Cluster_analysis',
 'Cluster_analysis_algorithms',
 'Clustering_criteria',
 'Computational_learning_theory',
 'Artificial_intelligence_conferences',
 'Signal_processing_conferences',
 'Data_mining_and_machine_learning_software',
 'Social_network_analysis_software',
 'Datasets_in_machine_learning',
 'Datasets_in_computer_vision',
 'Dimension_reduction',
 'Factor_analysis',
 'Evolutionary_algorithms',
 'Gene_expression_programming',
 'Genetic_algorithms',
 'Artificial_immune_systems',
 'Genetic_programming',
 'Nature-inspired_metaheuristics',
 'Inductive_logic_programming',
 'Kernel_methods_for_machine_learning',
 'Support_vector_machines',
 'Latent_variable_models',
 'Structural_equation_models',
 'Learning_in_computer_vision',
 'Log-linear_models',
 'Loss_

In [24]:
vectors

<1121x7034 sparse matrix of type '<class 'numpy.float64'>'
	with 86605 stored elements in Compressed Sparse Row format>

In [16]:
df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

In [17]:
df["article_name"] = article_names

In [18]:
#save to a file
df.to_hdf(fileName + "_embeddings.h5", key='df')

In [50]:
#read from file
data_table = pd.read_hdf("data/embeddings.h5", 'df')

In [12]:
data_table["article_name"]
data_table.index[data_table['article_name'] == "Julius_Caesar"]

Int64Index([316, 842, 887], dtype='int64')

In [19]:
data_table = pd.read_hdf(fileName + "_embeddings.h5", 'df')

In [20]:
data_table

,aaai,ab,ability,able,abstract,access,account,accuracy,achieve,acm,...,work,world,write,year,yield,zealand,zen,zero,zeta,article_name
0,0.000000,0.000000,0.024027,0.000000,0.000000,0.000000,0.000000,0.043990,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Machine_learning
1,0.000000,0.000000,0.024027,0.000000,0.000000,0.000000,0.000000,0.043990,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Applied_machine_learning
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032462,0.000000,...,0.000000,0.078130,0.000000,0.032462,0.000000,0.000000,0.028689,0.054090,0.000000,AlphaGo
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030781,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Artificial_neural_networks
4,0.000000,0.000000,0.021651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.023197,0.000000,0.000000,0.000000,0.000000,0.000000,Deep_learning
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Neural_network_software
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.022313,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Bayesian_networks
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.094730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Classification_algorithms
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Decision_trees
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057488,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.036505,0.000000,0.000000,0.000000,0.000000,Ensemble_learning


In [4]:
data_table.loc[1, "article_name"]

'Artificial_neural_networks'

In [21]:
data_table["article_name"]

0                              Machine_learning
1                      Applied_machine_learning
2                                       AlphaGo
3                    Artificial_neural_networks
4                                 Deep_learning
5                       Neural_network_software
6                             Bayesian_networks
7                     Classification_algorithms
8                                Decision_trees
9                             Ensemble_learning
10                             Cluster_analysis
11                  Cluster_analysis_algorithms
12                          Clustering_criteria
13                Computational_learning_theory
14          Artificial_intelligence_conferences
15                Signal_processing_conferences
16    Data_mining_and_machine_learning_software
17             Social_network_analysis_software
18                 Datasets_in_machine_learning
19                  Datasets_in_computer_vision
20                          Dimension_re

In [6]:
file = open(fileName + "_relations.txt", "r")
for line in file:
    print(line.rstrip("\n"))
file.close()

Artificial_neural_networks Machine_learning
Applied_machine_learning Machine_learning
Unsupervised_learning Machine_learning
Structured_prediction Machine_learning
Loss_functions Machine_learning
Support_vector_machines Machine_learning
Latent_variable_models Machine_learning
Cluster_analysis Machine_learning
Signal_processing_conferences Machine_learning
Artificial_intelligence_conferences Machine_learning
Data_mining_and_machine_learning_software Machine_learning
Machine_learning_algorithms Machine_learning
Bayesian_networks Machine_learning
Machine_learning_portal Machine_learning
Statistical_natural_language_processing Machine_learning
Markov_models Machine_learning
Evolutionary_algorithms Machine_learning
Genetic_programming Machine_learning
Datasets_in_machine_learning Machine_learning
Dimension_reduction Machine_learning
Machine_learning_researchers Machine_learning
Supervised_learning Machine_learning
Semisupervised_learning Machine_learning
Computational_learning_theory Machin